In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

C:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.11.4-CAPI-1.17.4) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'urban_centers', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by Regional Geography
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
df.rename(columns={'name': 'Regional Growth Center'}, inplace=True)
df['Regional Growth Center'] = df['Regional Growth Center'].fillna('Not in RGC')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,288,"3,990","9,006","39,231"
Bellevue,"1,168","8,870","18,480","65,267"
Bothell Canyon Park,"7,504","14,867","14,273","145,470"
Bremerton,66,698,"2,355","21,555"
Burien,179,"2,822","9,102","68,179"
Everett,165,"7,023","16,073","61,476"
Federal Way,85,"1,297","2,356","23,971"
Greater Downtown Kirkland,765,"7,015","14,848","116,097"
Issaquah,"2,261","7,016","8,369","175,880"


In [9]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.5%,7.6%,17.1%,74.7%
Bellevue,1.2%,9.5%,19.7%,69.6%
Bothell Canyon Park,4.1%,8.2%,7.8%,79.9%
Bremerton,0.3%,2.8%,9.5%,87.4%
Burien,0.2%,3.5%,11.3%,84.9%
Everett,0.2%,8.3%,19.0%,72.5%
Federal Way,0.3%,4.7%,8.5%,86.5%
Greater Downtown Kirkland,0.6%,5.1%,10.7%,83.7%
Issaquah,1.2%,3.6%,4.3%,90.9%


#### AM Peak (7 to 8 AM)

In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,26,334,"1,465","1,910"
Bellevue,66,613,"1,025","4,756"
Bothell Canyon Park,418,"2,591","1,839","6,352"
Bremerton,2,90,193,"1,897"
Burien,13,181,535,"3,949"
Everett,18,583,"1,033","4,000"
Federal Way,4,42,156,"1,329"
Greater Downtown Kirkland,92,"1,886","1,216","6,730"
Issaquah,287,484,"2,278","11,779"


In [11]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.7%,8.9%,39.2%,51.1%
Bellevue,1.0%,9.5%,15.9%,73.6%
Bothell Canyon Park,3.7%,23.1%,16.4%,56.7%
Bremerton,0.1%,4.1%,8.8%,87.0%
Burien,0.3%,3.9%,11.4%,84.4%
Everett,0.3%,10.3%,18.3%,71.0%
Federal Way,0.3%,2.8%,10.2%,86.8%
Greater Downtown Kirkland,0.9%,19.0%,12.3%,67.8%
Issaquah,1.9%,3.3%,15.4%,79.4%


#### PM Peak (5 to 6 PM)

In [12]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,22,472,705,"3,197"
Bellevue,114,"1,248","1,344","5,459"
Bothell Canyon Park,833,747,"1,604","10,245"
Bremerton,39,42,230,"1,943"
Burien,13,292,752,"5,753"
Everett,13,629,"1,320","5,068"
Federal Way,7,144,139,"1,791"
Greater Downtown Kirkland,79,797,"1,700","8,732"
Issaquah,504,703,920,"14,531"


In [13]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.5%,10.7%,16.0%,72.7%
Bellevue,1.4%,15.3%,16.5%,66.9%
Bothell Canyon Park,6.2%,5.6%,11.9%,76.3%
Bremerton,1.7%,1.9%,10.2%,86.2%
Burien,0.2%,4.3%,11.0%,84.5%
Everett,0.2%,8.9%,18.8%,72.1%
Federal Way,0.3%,6.9%,6.7%,86.1%
Greater Downtown Kirkland,0.7%,7.1%,15.0%,77.2%
Issaquah,3.0%,4.2%,5.5%,87.2%


# Network Delay
Totals scaled to match network totals. 

In [14]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
Regional Growth Center,
Auburn,125
Bellevue,231
Bothell Canyon Park,"1,058"
Bremerton,22
Burien,73
Everett,99
Federal Way,17
Greater Downtown Kirkland,376
Issaquah,543


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
Regional Growth Center,
Auburn,119
Bellevue,220
Bothell Canyon Park,"1,005"
Bremerton,21
Burien,70
Everett,94
Federal Way,16
Greater Downtown Kirkland,357
Issaquah,516
